# Iteration 0: unplanned arrivals - stage 2

## Plain English Summary

The first step in coding a `simpy` model is typically coding the arrivals to the system.  In the CCU model there are unplanned(emergency) versus planned (elective) arrivals.  Given the logic for unplanned arrivals is the simpler of the two types it is logical to code the arrival of these patients (5 subclasses) first.



## Notebook imports

In [23]:
from IPython.display import Markdown, display

## LLM Prompt

In [24]:
display(Markdown("../01_perplexity/inputs/iteration_0.md"))

### Main command

Code a discrete-event simulation model of a critical care unit (CCU) in python 3.10 and simpy 4. 

Please code the full model specified. Do not return a simplified version. Show all code.

### General model logic

All time units in the model are in hours.

Each patient in the model has a unique identifier.  The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.

Patients to arrive at the CCU from five different sources: Accident and Emergency, the Wards, Emergency surgery, other hospitals, or the X-Ray department. Each source had a different inter-arrival time distribution.    After patients arrive they immediately leave the model.

All patient types must have their own generator function

The model should print out useful information after each event.

the model should include a user settable run length. This defaults to 12 months.


### Simulation inputs

the interarrival time distributions and parameters of patients are dependent on patient type. For each distribution time is measured in hours.

Accident and Emergency = Exponential: 22.72
the Wards = Exponential: 26.0
Emergency surgery = Exponential: 37.0
other hospitals = Exponential: 47.2
the X-Ray department = Exponential: 575.0 


### simulation methodology

numpy should be use for sampling.

each interarrival distribution should have its own numpy.random.Generator object.

## Date and time of input to LLM

> 1130 30th May 2024

## Observations on the LLM Output for Stage 2

* RNGs are created in this iteration (they were not for Stage 1)
* Parameters are hard-coded (as with Stage 1)
* No CCU class is created
* A function is used to generate patients ids which is then used as a parameter in the generator functions
* The LLM made some reasonable assumptions about converting months into hours (assuming 30 days per month)
* Patient counts start from 0
* The model runs without error.

## Sources reported by the LLM

* No sources reported

## LLM Output_S2

This code sets up separate random number generators for each patient type using numpy.random.default_rng. It then defines five generator functions (accident_emergency_arrivals, wards_arrivals, emergency_surgery_arrivals, other_hospitals_arrivals, and xray_department_arrivals) that generate patient arrivals from the respective sources based on the specified exponential inter-arrival time distributions.
The ccu_simulation function initializes the simulation environment, starts the patient arrival processes, and runs the simulation for the specified duration (default: 12 months). The count function is a helper function that generates unique patient IDs.
After each patient arrival event, the code prints a message indicating the patient ID, source, and arrival time.
Note that this code does not include any additional logic for handling patients after they arrive, such as assigning beds or resources. It simply simulates the arrival process and prints the arrival information.

In [25]:
import simpy
import numpy as np

# Set up random number generators for each patient type
rng_accident_emergency = np.random.default_rng(seed=42)
rng_wards = np.random.default_rng(seed=43)
rng_emergency_surgery = np.random.default_rng(seed=44)
rng_other_hospitals = np.random.default_rng(seed=45)
rng_xray_department = np.random.default_rng(seed=46)

# Patient arrival generators
def accident_emergency_arrivals(env, patient_id_counter):
    while True:
        yield env.timeout(rng_accident_emergency.exponential(22.72))
        patient_id = next(patient_id_counter)
        print(f"Patient {patient_id} arrived from Accident and Emergency at {env.now:.2f} hours")

def wards_arrivals(env, patient_id_counter):
    while True:
        yield env.timeout(rng_wards.exponential(26.0))
        patient_id = next(patient_id_counter)
        print(f"Patient {patient_id} arrived from the Wards at {env.now:.2f} hours")

def emergency_surgery_arrivals(env, patient_id_counter):
    while True:
        yield env.timeout(rng_emergency_surgery.exponential(37.0))
        patient_id = next(patient_id_counter)
        print(f"Patient {patient_id} arrived from Emergency Surgery at {env.now:.2f} hours")

def other_hospitals_arrivals(env, patient_id_counter):
    while True:
        yield env.timeout(rng_other_hospitals.exponential(47.2))
        patient_id = next(patient_id_counter)
        print(f"Patient {patient_id} arrived from Other Hospitals at {env.now:.2f} hours")

def xray_department_arrivals(env, patient_id_counter):
    while True:
        yield env.timeout(rng_xray_department.exponential(575.0))
        patient_id = next(patient_id_counter)
        print(f"Patient {patient_id} arrived from the X-Ray Department at {env.now:.2f} hours")

# CCU simulation model
def ccu_simulation(env, run_length=12 * 30 * 24):  # Default run length: 12 months
    patient_id_counter = count()

    # Start patient arrival processes
    env.process(accident_emergency_arrivals(env, patient_id_counter))
    env.process(wards_arrivals(env, patient_id_counter))
    env.process(emergency_surgery_arrivals(env, patient_id_counter))
    env.process(other_hospitals_arrivals(env, patient_id_counter))
    env.process(xray_department_arrivals(env, patient_id_counter))

    # Run the simulation for the specified duration
    env.run(until=run_length)

# Helper function to generate unique patient IDs
def count():
    i = 0
    while True:
        yield i
        i += 1

# Run the simulation
env = simpy.Environment()
ccu_simulation(env)


Patient 0 arrived from the Wards at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 arrived from the Wards at 7.93 hours
Patient 3 arrived from Emergency Surgery at 13.71 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 arrived from Other Hospitals at 35.95 hours
Patient 6 arrived from the Wards at 38.90 hours
Patient 7 arrived from Accident and Emergency at 54.62 hours
Patient 8 arrived from Emergency Surgery at 76.27 hours
Patient 9 arrived from Other Hospitals at 77.00 hours
Patient 10 arrived from the Wards at 79.30 hours
Patient 11 arrived from the Wards at 89.81 hours
Patient 12 arrived from Accident and Emergency at 107.70 hours
Patient 13 arrived from Emergency Surgery at 116.09 hours
Patient 14 arrived from the Wards at 124.71 hours
Patient 15 arrived from the Wards at 127.85 hours
Patient 16 arrived from Emergency Surgery at 137.29 hours
Patient 17 arrived from Emergency Surgery at 140.62 hours
Patient 18 arrived from Other Hospitals

## Additional prompt:
Could you re-factor the coding into a CCU class 

In this refactored version, the CCU class encapsulates the entire simulation logic. The __init__ method initializes the simulation environment, sets up the random number generators, and starts the patient arrival processes.
The count method is a generator function that generates unique patient IDs.
The accident_emergency_arrivals, wards_arrivals, emergency_surgery_arrivals, other_hospitals_arrivals, and xray_department_arrivals methods are generator functions that simulate patient arrivals from their respective sources based on the specified exponential inter-arrival time distributions.
The run method runs the simulation for the specified duration.
To run the simulation, you create an instance of the CCU class and call its run method.
This refactored version follows the object-oriented programming paradigm and encapsulates the simulation logic within the CCU class, making the code more modular and easier to maintain or extend in the future.

In [26]:
import simpy
import numpy as np

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = self.count()

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

        # Start patient arrival processes
        env.process(self.accident_emergency_arrivals())
        env.process(self.wards_arrivals())
        env.process(self.emergency_surgery_arrivals())
        env.process(self.other_hospitals_arrivals())
        env.process(self.xray_department_arrivals())

    def count(self):
        i = 0
        while True:
            yield i
            i += 1

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            patient_id = next(self.patient_id_counter)
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            patient_id = next(self.patient_id_counter)
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            patient_id = next(self.patient_id_counter)
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            patient_id = next(self.patient_id_counter)
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            patient_id = next(self.patient_id_counter)
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")

    def run(self):
        self.env.run(until=self.run_length)

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()


Patient 0 arrived from the Wards at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 arrived from the Wards at 7.93 hours
Patient 3 arrived from Emergency Surgery at 13.71 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 arrived from Other Hospitals at 35.95 hours
Patient 6 arrived from the Wards at 38.90 hours
Patient 7 arrived from Accident and Emergency at 54.62 hours
Patient 8 arrived from Emergency Surgery at 76.27 hours
Patient 9 arrived from Other Hospitals at 77.00 hours
Patient 10 arrived from the Wards at 79.30 hours
Patient 11 arrived from the Wards at 89.81 hours
Patient 12 arrived from Accident and Emergency at 107.70 hours
Patient 13 arrived from Emergency Surgery at 116.09 hours
Patient 14 arrived from the Wards at 124.71 hours
Patient 15 arrived from the Wards at 127.85 hours
Patient 16 arrived from Emergency Surgery at 137.29 hours
Patient 17 arrived from Emergency Surgery at 140.62 hours
Patient 18 arrived from Other Hospitals

## Additional prompt:

Modify the model to initialise the patient_id_counter = 0

In this modified version, the patient_id_counter is initialized to 0 in the __init__ method of the CCU class. Additionally, in each of the patient arrival generator methods (accident_emergency_arrivals, wards_arrivals, emergency_surgery_arrivals, other_hospitals_arrivals, and xray_department_arrivals), the patient_id is assigned the current value of self.patient_id_counter, and then self.patient_id_counter is incremented by 1 after printing the arrival information. With this modification, the first patient to arrive will have an identifier of 0, and each subsequent patient will have an incremented identifier, as specified in the requirements.

In [27]:
import simpy
import numpy as np

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0  # Initialize patient_id_counter to 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

        # Start patient arrival processes
        env.process(self.accident_emergency_arrivals())
        env.process(self.wards_arrivals())
        env.process(self.emergency_surgery_arrivals())
        env.process(self.other_hospitals_arrivals())
        env.process(self.xray_department_arrivals())

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def run(self):
        self.env.run(until=self.run_length)

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()


Patient 0 arrived from the Wards at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 arrived from the Wards at 7.93 hours
Patient 3 arrived from Emergency Surgery at 13.71 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 arrived from Other Hospitals at 35.95 hours
Patient 6 arrived from the Wards at 38.90 hours
Patient 7 arrived from Accident and Emergency at 54.62 hours
Patient 8 arrived from Emergency Surgery at 76.27 hours
Patient 9 arrived from Other Hospitals at 77.00 hours
Patient 10 arrived from the Wards at 79.30 hours
Patient 11 arrived from the Wards at 89.81 hours
Patient 12 arrived from Accident and Emergency at 107.70 hours
Patient 13 arrived from Emergency Surgery at 116.09 hours
Patient 14 arrived from the Wards at 124.71 hours
Patient 15 arrived from the Wards at 127.85 hours
Patient 16 arrived from Emergency Surgery at 137.29 hours
Patient 17 arrived from Emergency Surgery at 140.62 hours
Patient 18 arrived from Other Hospitals

## Additional prompt

Modify the model so that all patient arrival processes are inside the 'run' function. Show the full code listing. Do not hide any functions or classes.

In this modified version, the patient arrival processes are no longer started in the __init__ method. Instead, they are started inside the run method.

The env.process method is used to start each patient arrival process, and the resulting process objects are stored in variables (accident_emergency_process, wards_process, emergency_surgery_process, other_hospitals_process, and xray_department_process). However, these variables are not used further in the code, as the processes run independently once started.
The self.env.run(until=self.run_length) line runs the simulation for the specified duration.
All other parts of the code remain the same, including the patient arrival generator functions (accident_emergency_arrivals, wards_arrivals, emergency_surgery_arrivals, other_hospitals_arrivals, and xray_department_arrivals), and the CCU class definition.

In [28]:
import simpy
import numpy as np
from scipy.stats import lognorm

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # Lognormal parameters
        self.accident_emergency_lognormal_mu = 128.79
        self.accident_emergency_lognormal_sigma = 267.51
        self.wards_lognormal_mu = 177.89
        self.wards_lognormal_sigma = 276.54
        self.emergency_surgery_lognormal_mu = 140.15
        self.emergency_surgery_lognormal_sigma = 218.02
        self.other_hospitals_lognormal_mu = 212.86
        self.other_hospitals_lognormal_sigma = 457.67
        self.xray_department_lognormal_mu = 87.53
        self.xray_department_lognormal_sigma = 108.67

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.accident_emergency_lognormal_mu, self.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.wards_lognormal_mu, self.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.emergency_surgery_lognormal_mu, self.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.other_hospitals_lognormal_mu, self.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:  # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.xray_department_lognormal_mu, self.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(5)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())

        self.env.run(until=self.run_length)

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()



Patient 0 arrived from the Wards at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 arrived from the Wards at 7.93 hours
Patient 3 arrived from Emergency Surgery at 13.71 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 arrived from Other Hospitals at 35.95 hours
Patient 6 arrived from the Wards at 38.90 hours
Patient 7 arrived from Accident and Emergency at 54.62 hours
Patient 8 arrived from Emergency Surgery at 76.27 hours
Patient 9 arrived from Other Hospitals at 77.00 hours
Patient 10 arrived from the Wards at 79.30 hours
Patient 11 arrived from the Wards at 89.81 hours
Patient 12 arrived from Accident and Emergency at 107.70 hours
Patient 13 arrived from Emergency Surgery at 116.09 hours
Patient 14 arrived from the Wards at 124.71 hours
Patient 15 arrived from the Wards at 127.85 hours
Patient 16 arrived from Emergency Surgery at 137.29 hours
Patient 17 arrived from Emergency Surgery at 140.62 hours
Patient 18 arrived from Other Hospitals

## Testing

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [29]:
M = 10_000_000

In [30]:
class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())

        self.env.run(until=self.run_length)

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()

Patient 0 arrived from Accident and Emergency at 54.62 hours
Patient 1 arrived from Accident and Emergency at 107.70 hours
Patient 2 arrived from Accident and Emergency at 161.88 hours
Patient 3 arrived from Accident and Emergency at 168.24 hours
Patient 4 arrived from Accident and Emergency at 170.20 hours
Patient 5 arrived from Accident and Emergency at 203.21 hours
Patient 6 arrived from Accident and Emergency at 235.24 hours
Patient 7 arrived from Accident and Emergency at 306.23 hours
Patient 8 arrived from Accident and Emergency at 308.03 hours
Patient 9 arrived from Accident and Emergency at 331.81 hours
Patient 10 arrived from Accident and Emergency at 333.41 hours
Patient 11 arrived from Accident and Emergency at 358.15 hours
Patient 12 arrived from Accident and Emergency at 397.49 hours
Patient 13 arrived from Accident and Emergency at 406.28 hours
Patient 14 arrived from Accident and Emergency at 434.26 hours
Patient 15 arrived from Accident and Emergency at 437.75 hours
Pat

### Extreme value test 2

* The code is again modified so that Accident and Emergency patients also have a mean inter-arrival rate of $M$

**Result: PASS**

In [31]:
class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(M))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())

        self.env.run(until=self.run_length)

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()

### Different run length test

* The code is again modified to use the original parameters generated by the LLM.

**Result: PASS**

In [32]:
import simpy
import numpy as np

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            patient_id = self.patient_id_counter
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())

        self.env.run(until=self.run_length)

In [33]:
TEN_DAYS = 24 * 10

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu = CCU(env, run_length=TEN_DAYS)

ccu.run()

Patient 0 arrived from the Wards at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 arrived from the Wards at 7.93 hours
Patient 3 arrived from Emergency Surgery at 13.71 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 arrived from Other Hospitals at 35.95 hours
Patient 6 arrived from the Wards at 38.90 hours
Patient 7 arrived from Accident and Emergency at 54.62 hours
Patient 8 arrived from Emergency Surgery at 76.27 hours
Patient 9 arrived from Other Hospitals at 77.00 hours
Patient 10 arrived from the Wards at 79.30 hours
Patient 11 arrived from the Wards at 89.81 hours
Patient 12 arrived from Accident and Emergency at 107.70 hours
Patient 13 arrived from Emergency Surgery at 116.09 hours
Patient 14 arrived from the Wards at 124.71 hours
Patient 15 arrived from the Wards at 127.85 hours
Patient 16 arrived from Emergency Surgery at 137.29 hours
Patient 17 arrived from Emergency Surgery at 140.62 hours
Patient 18 arrived from Other Hospitals